In [1]:
#installing dependencies
!pip install -q transformers datasets peft bitsandbytes accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 34.8 MB/s eta 0:00:00:00:0100:01


In [2]:
#imports
import warnings
warnings.filterwarnings("ignore")
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model
from datasets import Dataset
from peft import PeftModel


2026-01-30 07:06:12.580786: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1769756772.722681      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1769756772.762272      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1769756773.083140      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769756773.083177      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769756773.083179      55 computation_placer.cc:177] computation placer alr

In [3]:
#Loading the model 
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,
    device_map="auto"
)


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [4]:
#lora-config for fine-tuning
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


trainable params: 1,126,400 || all params: 1,101,174,784 || trainable%: 0.1023


In [5]:
#importing the datasets for finetuning
import pandas as pd
df = pd.read_csv("/kaggle/input/imdb-movies-dataset/imdb_movies.csv")
df.head()


data = []

for _, row in df.iterrows():
    title = row["names"]
    overview = row["overview"]
    genre = row["genre"]
    score = row["score"]

    # Q1: plot explanation
    q1 = f"What is the movie {title} about?"
    a1 = overview

    data.append({
        "text": f"### Question: {q1}\n### Answer: {a1}"
    })

    # Q2: genre-based recommendation
    q2 = f"Recommend a {genre} movie."
    a2 = f"{title} is a good recommendation. It has a user score of {score}. {overview}"

    data.append({
        "text": f"### Question: {q2}\n### Answer: {a2}"
    })



In [6]:
#converting datasets into HuggingFace datasets 
from datasets import Dataset

# convert list to dataset first if not already
full_dataset = Dataset.from_list(data)

# shuffle + take subset
dataset = full_dataset.shuffle(seed=42).select(range(6000))

len(dataset)


6000

In [7]:
def tokenize(example):
    tokens = tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=256
    )
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens

dataset = dataset.map(tokenize, batched=False)
dataset = dataset.remove_columns(["text"])
dataset.set_format("torch")


Map:   0%|          | 0/6000 [00:00<?, ? examples/s]

In [ ]:
#Training args 
training_args = TrainingArguments(
    output_dir="./movie-llm",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    num_train_epochs=3,
    fp16=True,

    logging_strategy="steps",   # 👈 force logging
    logging_steps=1,            # 👈 every step
    logging_first_step=True,

    disable_tqdm=True,          # 👈 KILLS progress bar (important)
    report_to="none",           # 👈 no wandb / tensorboard
    save_strategy="no",
    log_level="info"
)


In [ ]:
#Fine tuning the model 
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset
)
print("🚀 TRAINING STARTED")
trainer.train()
print("✅ TRAINING FINISHED")


In [ ]:
#saving the finetuned model 
trainer.model.save_pretrained("movie-lora-adapter")
tokenizer.save_pretrained("movie-lora-adapter")


After saving the finetuned model we are loading that model as LORA_PATH and testing it.

In [ ]:
!ls /kaggle/input/d/jeevaaaa/movie-lora/movie-lora-adapter


In [ ]:
!unzip /kaggle/input/movie-lora-adapter/movie-lora-adapter.zip


In [ ]:
!ls movie-lora-adapter


In [8]:
#using the finetuned model 
LORA_PATH = "Jeevaaaa/movie-lora-adapter"

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

model = PeftModel.from_pretrained(
    model,
    LORA_PATH
)

model.eval()


adapter_config.json:   0%|          | 0.00/868 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/4.52M [00:00<?, ?B/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): PeftModelForCausalLM(
      (base_model): LoraModel(
        (model): LlamaForCausalLM(
          (model): LlamaModel(
            (embed_tokens): Embedding(32000, 2048)
            (layers): ModuleList(
              (0-21): 22 x LlamaDecoderLayer(
                (self_attn): LlamaAttention(
                  (q_proj): lora.Linear4bit(
                    (base_layer): Linear4bit(in_features=2048, out_features=2048, bias=False)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.05, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=2048, out_features=8, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_features=8, out_features=2048, bias=False)
                    )
                    (lora_embedding_A): ParameterDict()
              

In [9]:
#testing the finetuned model 
prompt = """### Question: Recommend a science fiction movie with a twist.
### Answer:"""

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=120,
        do_sample=True,          # 👈 IMPORTANT
        temperature=0.4,         # low = controlled
        top_p=0.9,
        repetition_penalty=1.1,
        eos_token_id=tokenizer.eos_token_id,
    )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))


### Question: Recommend a science fiction movie with a twist.
### Answer: "The Martians" (2017)

"The Martians" is a science fiction movie that tells the story of a group of astronauts who land on Mars and discover that it has been colonized by an alien race, led by a mysterious and powerful being known as the Martian King. The film follows their attempts to establish contact and communication with the Martians, but they soon realize that they are not alone on the planet.

The twist in this movie is that the Martians are not hostile or aggressive, but instead,
